# Music

## Analyze Spotify playlists tracks

In [42]:
import os
import pandas as pd
from pandasql import sqldf as sql_df

data_path = os.getcwd().replace("\\", "/")[:-6] + "data/" # 6 = len("assets")
spotify_playlists_tracks = pd.read_csv(data_path + '240406SpotifyPlaylistsTracks.tsv', sep='\t', header=0)
# spotify_playlists_tracks[:1].T
# 	added_at          	2018-12-30 01:31:34
# 	added_by          	darkjavier
# 	album_name        	The Brain Dance
# 	album_release_date	2016-09-30
# 	album_n_tracks    	1.0
# 	artists_name      	Animals As Leaders
# 	duration_ms       	420746
# 	id                	5yKbTecdbmBikj6UbZDEhX
# 	name              	The Brain Dance
# 	popularity        	0
# 	track_number      	1
# 	playlist_name     	progressive rock
spotify_playlists_tracks[["playlist_name","artists_name","name","id"]][:2].T

,0,1
playlist_name,Likes,Likes
artists_name,Ane Brun,Vince Clarke
name,Fly On The Windscreen,Fly On The Windscreen
id,5jHoirLJZh35JMHIaru2qL,5jHoirLJZh35JMHIaru2qL


In [43]:
spotify_duplicates = sql_df('''
  SELECT artists_name, name as track_name, id as track_id, count(*) as n_duplicates
  FROM spotify_playlists_tracks
  GROUP BY artists_name, track_name, track_id
  HAVING n_duplicates>1
  ORDER BY n_duplicates DESC, artists_name ASC, track_name ASC
''')
spotify_duplicates

,artists_name,track_name,track_id,n_duplicates
0,65daysofstatic,Debutante,0CpaNwilKJGqn9WF6K6aRV,5
1,Amy Macdonald,This Is The Life,3oyqGFcN7m3jk4alkNI3j9,5
2,Anan Ryoko,Refrain,0bXZImFaGuFsAyszjQ5DSG,5
3,Camille Saint-Saëns,Aquarium,2IBLeC7kvXtO7iZHGTCYfm,5
4,Dance With the Dead,Get Out,2yrh2iC9s9wXbnc9fSCuIn,5
...,...,...,...,...
1767,zircon,The End,24HZkXA56U0OUrkJ8fgzpv,2
1768,Ácido Pantera,Al Son Que Me Toquen Bailo,1qaHgJiapwLuWlg1uFQIWC,2
1769,Édith Piaf,"Non, je ne regrette rien",0D4gIeyrMtxOAvCWsTHN9x,2
1770,Šventinis bankuchenas,Vasaraš Atostogaš,7HWW5fbqCyKZZNajEaUxxG,2


In [44]:
songs_per_playlist = sql_df("""
SELECT playlist_name, count(*) as n_tracks
FROM spotify_playlists_tracks
GROUP BY playlist_name
ORDER BY playlist_name ASC
""") # LIMIT 6
songs_per_playlist.to_csv("../data/songs_per_playlist.csv")
songs_per_playlist

,playlist_name,n_tracks
0,Likes,420
1,LotR chronologically,37
2,Menuo Juodaragis 2022,147
3,acoust&ballads&sg-sw 10s and beyond,165
4,acoust&ballads&sg-sw/00s,226
...,...,...
68,tribal,85
69,trip-hop,77
70,uplifting ost,307
71,woods and folk,192


In [45]:
duplicates_per_playlist = sql_df("""
SELECT DISTINCT spt.playlist_name, COUNT(sd.n_duplicates) AS n_duplicates, spp.n_tracks
FROM spotify_playlists_tracks as spt
INNER JOIN spotify_duplicates AS sd
ON spt.id = sd.track_id
INNER JOIN songs_per_playlist as spp
ON spt.playlist_name = spp.playlist_name
GROUP BY 1, 3
ORDER BY spt.playlist_name ASC
""")
percentage_duplicates_per_playlist = sql_df("""
SELECT playlist_name, min(100*n_duplicates/n_tracks, 100) AS percentage_duplicates
FROM duplicates_per_playlist
ORDER BY percentage_duplicates DESC
""")
percentage_duplicates_per_playlist.to_csv("../data/percentage_duplicates_per_playlist.csv")
percentage_duplicates_per_playlist

,playlist_name,percentage_duplicates
0,Likes,100
1,covers mainstream (less rock),100
2,instrumental glory,100
3,just covers ( ͡° ͜ʖ ͡° ),100
4,snow,100
...,...,...
67,Menuo Juodaragis 2022,7
68,neofolk mainstream,5
69,kizomba,4
70,salsa,4


In [47]:
duplicated_tracks_per_playlist = sql_df("""
SELECT DISTINCT spt.playlist_name, sd.artists_name, sd.track_name
FROM spotify_playlists_tracks as spt
INNER JOIN spotify_duplicates AS sd
ON spt.id = sd.track_id
ORDER BY 1, 2, 3
""")
duplicated_tracks_per_playlist.to_csv("../data/duplicated_tracks_per_playlist.csv")
duplicated_tracks_per_playlist = sql_df("""
SELECT *
FROM duplicated_tracks_per_playlist
ORDER BY 2, 3, 1
""")

,playlist_name,artists_name,track_name
0,Likes,1200 Micrograms,Acid For Nothing - Original Mix
1,Likes,2 Times Terror,Ikävässä Paikasa
2,Likes,5'nizza,Sjurnaya
3,Likes,65daysofstatic,Debutante
4,Likes,69 Danguje,Velniškas Greitis
...,...,...,...
4024,workout,Vini Vici,Colors
4025,workout,Wizard Lizard,Beat Crashers
4026,workout,XSI,Nightmare - Talamasca vs XSI ft Lucid Remix
4027,workout,Zyce,Shiva - Original Mix
